In [1]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import re

rootURL = 'https://www.pro-football-reference.com/'
playerURL_list = []
player_list = []

ppr = 1

# HTML cleaner
def _strip_html(text):
    tag_re = re.compile(r'<[^>]+>')
    return tag_re.sub('', str(text))

url = 'https://www.pro-football-reference.com/years/2020/scrimmage.htm'
## Scrape for RB stats
res = requests.get(url, verify=False)
soup = BeautifulSoup(res.text,features="html.parser")

parsed = soup.findAll(
    'table', {
        'id': 'receiving_and_rushing'
    }
)

rows = parsed[0].findAll('td')

column_len = 30

grouped_rows = [
    rows[i:i+column_len] for i in range(0, len(rows), column_len)
]


for i in grouped_rows:
    fullSTR = str(i)
    trimmed = re.search('href="(.*).htm">',fullSTR)
    playerURL = trimmed.group(1)
    playerURL = rootURL + playerURL + '.htm'
    playerURL_list.append(playerURL)
    
table = [map(lambda x: _strip_html(x), row) for row in grouped_rows]
    
for i in table:
    player_list.append(list(i))

C:\Users\lapet\AppData\Local\Programs\Python\Python38-32\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.pro-football-reference.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [2]:
import pandas as pd

In [3]:
url = playerURL_list[0]

player_data = pd.read_html(url, header=1)

In [4]:
player_info = player_data[0].ffill().bfill().iloc[:-1, :]
player_info = player_info.apply(lambda x: x.astype(str).str.upper())

In [5]:
player_info.columns

Index(['Year', 'Age', 'Tm', 'Pos', 'No.', 'G', 'GS', 'Rush', 'Yds', 'TD', '1D',
       'Lng', 'Y/A', 'Y/G', 'A/G', 'Tgt', 'Rec', 'Yds.1', 'Y/R', 'TD.1',
       '1D.1', 'Lng.1', 'R/G', 'Y/G.1', 'Ctch%', 'Y/Tgt', 'Touch', 'Y/Tch',
       'YScm', 'RRTD', 'Fmb', 'AV'],
      dtype='object')

In [6]:
player_info['Year'] = player_info['Year'].str.replace('+', '').str.replace('*', '')
player_info['Ctch%'] = player_info['Ctch%'].str.replace('%', '')

In [7]:
column_names = ['Year', 'Age', 'Team', 'Pos', 'No', 'Games', 'GameStarted', 'RushAtts', 'RushYds', 'RushTD', 'Rush1D', 'RushLong', 'YdsRush', 'YdsGame', 'RushAttGame', 'RecTarget', 'Receptions', 'RecYds', 'YdsRec', 'RecTD', 'Rec1D', 'RecLong', 'RecGame', 'RecYdsGame', 'CatchPerc', 'YdsTarget', 'Touches', 'YdsTouch', 'ScrimYds', 'RushRecTD', 'Fumbles', 'ApproxValue']

In [8]:
player_info.columns = column_names

In [9]:
player_info

,Year,Age,Team,Pos,No,Games,GameStarted,RushAtts,RushYds,RushTD,...,RecGame,RecYdsGame,CatchPerc,YdsTarget,Touches,YdsTouch,ScrimYds,RushRecTD,Fumbles,ApproxValue
0,2016,22,TEN,RB,22.0,15,2,110,490,5,...,0.9,9.1,86.7,9.1,123,5.1,627,5,0,4
1,2017,23,TEN,RB,22.0,16,2,176,744,5,...,0.7,8.5,64.7,8.0,187,4.7,880,6,1,6
2,2018,24,TEN,RB,22.0,16,12,215,1059,12,...,0.9,6.2,83.3,5.5,230,5.0,1158,12,1,7
3,2019,25,TEN,RB,22.0,15,15,303,1540,16,...,1.2,13.7,75.0,8.6,321,5.4,1746,18,5,13
4,2020,26,TEN,RB,22.0,16,16,378,2027,17,...,1.2,7.1,61.3,3.7,397,5.4,2141,17,3,17


In [10]:
# Scoring:
# 0.1 / rec yd
# 1 / reception
# 6 / rec td
# 0.1 / rush yd
# 6 / rush td

In [11]:
player_info[player_info.columns[5:]] = player_info[player_info.columns[5:]].astype(float)

In [12]:
player_info['TotScore'] = (player_info['RushRecTD'] * 6) + (player_info['ScrimYds'] * 0.1) + (player_info['Receptions'] * ppr)

In [13]:
player_info

,Year,Age,Team,Pos,No,Games,GameStarted,RushAtts,RushYds,RushTD,...,RecYdsGame,CatchPerc,YdsTarget,Touches,YdsTouch,ScrimYds,RushRecTD,Fumbles,ApproxValue,TotScore
0,2016,22,TEN,RB,22.0,15.0,2.0,110.0,490.0,5.0,...,9.1,86.7,9.1,123.0,5.1,627.0,5.0,0.0,4.0,105.7
1,2017,23,TEN,RB,22.0,16.0,2.0,176.0,744.0,5.0,...,8.5,64.7,8.0,187.0,4.7,880.0,6.0,1.0,6.0,135.0
2,2018,24,TEN,RB,22.0,16.0,12.0,215.0,1059.0,12.0,...,6.2,83.3,5.5,230.0,5.0,1158.0,12.0,1.0,7.0,202.8
3,2019,25,TEN,RB,22.0,15.0,15.0,303.0,1540.0,16.0,...,13.7,75.0,8.6,321.0,5.4,1746.0,18.0,5.0,13.0,300.6
4,2020,26,TEN,RB,22.0,16.0,16.0,378.0,2027.0,17.0,...,7.1,61.3,3.7,397.0,5.4,2141.0,17.0,3.0,17.0,335.1


In [14]:
player_info.insert(0, column='Name', value=player_list[0][0].replace('*','').replace('+',''))

In [15]:
player_info

,Name,Year,Age,Team,Pos,No,Games,GameStarted,RushAtts,RushYds,...,RecYdsGame,CatchPerc,YdsTarget,Touches,YdsTouch,ScrimYds,RushRecTD,Fumbles,ApproxValue,TotScore
0,Derrick Henry,2016,22,TEN,RB,22.0,15.0,2.0,110.0,490.0,...,9.1,86.7,9.1,123.0,5.1,627.0,5.0,0.0,4.0,105.7
1,Derrick Henry,2017,23,TEN,RB,22.0,16.0,2.0,176.0,744.0,...,8.5,64.7,8.0,187.0,4.7,880.0,6.0,1.0,6.0,135.0
2,Derrick Henry,2018,24,TEN,RB,22.0,16.0,12.0,215.0,1059.0,...,6.2,83.3,5.5,230.0,5.0,1158.0,12.0,1.0,7.0,202.8
3,Derrick Henry,2019,25,TEN,RB,22.0,15.0,15.0,303.0,1540.0,...,13.7,75.0,8.6,321.0,5.4,1746.0,18.0,5.0,13.0,300.6
4,Derrick Henry,2020,26,TEN,RB,22.0,16.0,16.0,378.0,2027.0,...,7.1,61.3,3.7,397.0,5.4,2141.0,17.0,3.0,17.0,335.1


In [16]:
# attempt to create aggregations

In [17]:
player_info.columns

Index(['Name', 'Year', 'Age', 'Team', 'Pos', 'No', 'Games', 'GameStarted',
       'RushAtts', 'RushYds', 'RushTD', 'Rush1D', 'RushLong', 'YdsRush',
       'YdsGame', 'RushAttGame', 'RecTarget', 'Receptions', 'RecYds', 'YdsRec',
       'RecTD', 'Rec1D', 'RecLong', 'RecGame', 'RecYdsGame', 'CatchPerc',
       'YdsTarget', 'Touches', 'YdsTouch', 'ScrimYds', 'RushRecTD', 'Fumbles',
       'ApproxValue', 'TotScore'],
      dtype='object')

In [24]:
cols = ['Games', 'RushAtts', 'RushYds', 'RushTD', 'Rush1D', 'RecTarget', 'Receptions', 'RecYds', 'RecTD', 'Rec1D', 'Touches', 'ScrimYds', 'TotScore']

In [26]:
len(player_info['Year'].iloc[-1])

4

In [96]:
player_season_avgs = player_info.iloc[:-1, :][cols]

In [97]:
player_season_avgs

,Games,RushAtts,RushYds,RushTD,Rush1D,RecTarget,Receptions,RecYds,RecTD,Rec1D,Touches,ScrimYds,TotScore
0,15.0,110.0,490.0,5.0,29.0,15.0,13.0,137.0,0.0,5.0,123.0,627.0,105.7
1,16.0,176.0,744.0,5.0,40.0,17.0,11.0,136.0,1.0,4.0,187.0,880.0,135.0
2,16.0,215.0,1059.0,12.0,51.0,18.0,15.0,99.0,0.0,4.0,230.0,1158.0,202.8
3,15.0,303.0,1540.0,16.0,73.0,24.0,18.0,206.0,2.0,4.0,321.0,1746.0,300.6


In [98]:
stats = []
stats_df = pd.DataFrame(columns=player_season_avgs.columns[1:])

for i in range(0, len(player_season_avgs)):
    for c in player_season_avgs.columns[1:]:
        games = player_season_avgs['Games'].sum()
        stats.append(player_season_avgs[c].sum() / games)
        print(games, player_season_avgs[c].sum(), player_season_avgs[c].sum() / games)

    stats_df.loc[len(stats_df)] = stats
    player_season_avgs = player_season_avgs.iloc[:-1, :]
    stats = []

62.0 804.0 12.96774193548387
62.0 3833.0 61.82258064516129
62.0 38.0 0.6129032258064516
62.0 193.0 3.1129032258064515
62.0 74.0 1.1935483870967742
62.0 57.0 0.9193548387096774
62.0 578.0 9.32258064516129
62.0 3.0 0.04838709677419355
62.0 17.0 0.27419354838709675
62.0 861.0 13.887096774193548
62.0 4411.0 71.14516129032258
62.0 744.1 12.001612903225807
47.0 501.0 10.659574468085106
47.0 2293.0 48.787234042553195
47.0 22.0 0.46808510638297873
47.0 120.0 2.5531914893617023
47.0 50.0 1.0638297872340425
47.0 39.0 0.8297872340425532
47.0 372.0 7.914893617021277
47.0 1.0 0.02127659574468085
47.0 13.0 0.2765957446808511
47.0 540.0 11.48936170212766
47.0 2665.0 56.702127659574465
47.0 443.5 9.436170212765957
31.0 286.0 9.225806451612904
31.0 1234.0 39.806451612903224
31.0 10.0 0.3225806451612903
31.0 69.0 2.225806451612903
31.0 32.0 1.032258064516129
31.0 24.0 0.7741935483870968
31.0 273.0 8.806451612903226
31.0 1.0 0.03225806451612903
31.0 9.0 0.2903225806451613
31.0 310.0 10.0
31.0 1507.0 48.6

In [99]:
stats_df

,RushAtts,RushYds,RushTD,Rush1D,RecTarget,Receptions,RecYds,RecTD,Rec1D,Touches,ScrimYds,TotScore
0,12.967742,61.822581,0.612903,3.112903,1.193548,0.919355,9.322581,0.048387,0.274194,13.887097,71.145161,12.001613
1,10.659574,48.787234,0.468085,2.553191,1.063830,0.829787,7.914894,0.021277,0.276596,11.489362,56.702128,9.436170
2,9.225806,39.806452,0.322581,2.225806,1.032258,0.774194,8.806452,0.032258,0.290323,10.000000,48.612903,7.764516
3,7.333333,32.666667,0.333333,1.933333,1.000000,0.866667,9.133333,0.000000,0.333333,8.200000,41.800000,7.046667


In [91]:
player_season_avgs['Games'].sum()

62.0

In [92]:
player_season_avgs['RushYds'].sum()

3833.0

In [93]:
3833/62

61.82258064516129

In [94]:
player_season_avgs.iloc[:-1, :]

,Games,RushAtts,RushYds,RushTD,Rush1D,RecTarget,Receptions,RecYds,RecTD,Rec1D,Touches,ScrimYds,TotScore
0,15.0,110.0,490.0,5.0,29.0,15.0,13.0,137.0,0.0,5.0,123.0,627.0,105.7
1,16.0,176.0,744.0,5.0,40.0,17.0,11.0,136.0,1.0,4.0,187.0,880.0,135.0
2,16.0,215.0,1059.0,12.0,51.0,18.0,15.0,99.0,0.0,4.0,230.0,1158.0,202.8
